In [2]:
#Installing the necessary libraries
!pip install requests
!pip install beautifulsoup4
!pip install nltk



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/41.5 kB ? eta -:--:--
     --------- ------------------------------ 10.2/41.5 kB ? eta -:--:--
     ---------------------------- --------- 30.7/41.5 kB 330.3 kB/s eta 0:00:01
     -------------------------------------- 41.5/41.5 kB 333.1 kB/s eta 0:00:00
     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ---------------------------------------- 57.6/57.6 kB 1.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   - -------------------------------------- 0.1/1.5 MB 3.8 MB/s eta 0:00:01
   -- ------------------------------------- 0.1/1.5 MB 2.6 MB/s eta 0:00:01
   ---- ----------------------------------- 0.2/1.5 MB 1.5 MB/s eta 0:00:01
   ------ --------------------------------- 0.2/1.5 MB 1.4 MB/s eta 0:00:01
   ------- -------------------------------- 0.3/1.5 MB 1.5 MB/s eta 0:00:01
   ---------- ----------------------------- 0.4/1.5 MB 1.6 MB/s eta 0:00:01
   ------------ --


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
#Importing the library functionalities
import re
import requests
from bs4 import BeautifulSoup
import time
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import urllib.robotparser

In [4]:
# Function to fetch the HTML content of a page
def fetch_page(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.text
        else:
            print(f"Failed to fetch {url}: Status code {response.status_code}")
            return None
    except Exception as e:
        print(f"Error fetching {url}: {e}")
        return None

In [5]:
# Function to extract all links from a webpage
def extract_links(html_content, base_url):
    soup = BeautifulSoup(html_content, 'html.parser')
    links = set()  # Using a set to avoid duplicate URLs
    for anchor in soup.find_all('a', href=True):
        href = anchor['href']
        if href.startswith('http'):  # Ensure it's a complete URL
            links.add(href)
        else:
            links.add(base_url + href)  # Handle relative URLs
    return links

In [6]:
# Function to check if crawling a URL is allowed by robots.txt
def can_crawl(url, user_agent="*"):
    # Create a RobotFileParser object
    rp = urllib.robotparser.RobotFileParser()
    
    # Get the robots.txt URL by adding /robots.txt to the base domain
    base_url = url.split("/")[0] + "//" + url.split("/")[2]
    robots_txt_url = base_url + "/robots.txt"
    
    try:
        rp.set_url(robots_txt_url)
        rp.read()
        return rp.can_fetch(user_agent, url)  # Check if the URL can be fetched
    except Exception as e:
        print(f"Could not read robots.txt for {url}: {e}")
        return True  # Allow crawling by default if robots.txt is not accessible


In [18]:
# Main crawler function
def crawl(seed_url, max_pages=10):
    to_crawl = [seed_url]  # List of pages to visit
    crawled = set()  # Set of pages already visited

    while to_crawl and len(crawled) < max_pages:
        url = to_crawl.pop(0)  # Get the next page to crawl
        if url in crawled:
            continue  # Skip if already crawled

        # Check if the URL is allowed to be crawled by robots.txt
        if not can_crawl(url):
            print(f"Blocked by robots.txt: {url}")
            continue

        print(f"Crawling: {url}")
        html_content = fetch_page(url)

        if html_content:
            links = extract_links(html_content, url)
            to_crawl.extend(links - crawled)  # Add new links to crawl
            crawled.add(url)

        time.sleep(0.4)  # Add a delay to avoid overwhelming servers

    print(f"Crawled {len(crawled)} pages")

# Start crawling from a seed URL
seed_url = "https://jasper-johansen.github.io/julia_documentation/"
crawl(seed_url)

Crawling: https://jasper-johansen.github.io/julia_documentation/
Crawling: https://jasper-johansen.github.io/julia_documentation/#Introduction
Crawling: https://jasper-johansen.github.io/julia_documentation/#Variables_and_Constants
Crawling: https://docs.julialang.org/en/v1/
Crawling: https://jasper-johansen.github.io/julia_documentation/#References
Crawling: https://jasper-johansen.github.io/julia_documentation/#Datatypes_and_Operations
Crawling: https://julialang.org/downloads/
Crawling: https://jasper-johansen.github.io/julia_documentation/#Comments
Crawling: https://jasper-johansen.github.io/julia_documentation/#Control_Statements
Crawling: https://jasper-johansen.github.io/julia_documentation/#Getting_Started
Crawled 10 pages
